In [3]:
#importing packages needed
import requests
import pandas as pd
import json

### Accessing User Data File 

Because this is a project showing the possibilities, our user data is generated from an API instead of a file export from a CRM. 

In [33]:
#accessing the API to get user data
r = requests.get('https://peoplegeneratorapi.live/api/person/10')
print(r.status_code)
print(r.json())

200
[{'name': 'Rafael Kilback', 'age': 52, 'job': 'Journalist', 'incomeUSD': 68750, 'creditScore': 475, 'ccNumber': '3767-819973-23952', 'married': True, 'hasChildren': True, 'height': 161.0, 'weight': 54.8, 'eyeColor': 'GRAY', 'email': 'rafaelkilback@hotmail.com', 'gender': 'Female', 'hasDegree': True, 'bloodType': 'A+', 'username': 'rafael91', 'politicalLeaning': -0.02, 'religion': 'Buddhism', 'address': {'streetAddress': '726 Hartmann Dale', 'city': None, 'state': None, 'country': "Cote d'Ivoire", 'zip': '1392', 'geonameId': -1, 'phoneNumber': '+225 007808377', 'ipAddress': '125.81.113.10', 'countryCode': 'CI'}, 'doB': 'Sun Jun 11 09:38:07 UTC 1972', 'gpa': 3.2}, {'name': 'Lise Bartoletti', 'age': 24, 'job': 'Jockey', 'incomeUSD': 34522, 'creditScore': 704, 'ccNumber': '6767-5068-4665-5264-425', 'married': True, 'hasChildren': True, 'height': 165.0, 'weight': 64.1, 'eyeColor': 'BLUE', 'email': 'lise_bartoletti@hotmail.com', 'gender': 'Female', 'hasDegree': False, 'bloodType': 'B-', 

In [34]:
#viewing the data as a dataframe
data=r.json()
df = (pd.DataFrame(data))
df.head()

,name,age,job,incomeUSD,creditScore,ccNumber,married,hasChildren,height,weight,...,email,gender,hasDegree,bloodType,username,politicalLeaning,religion,address,doB,gpa
0,Rafael Kilback,52,Journalist,68750,475,3767-819973-23952,True,True,161.0,54.8,...,rafaelkilback@hotmail.com,Female,True,A+,rafael91,-0.02,Buddhism,"{'streetAddress': '726 Hartmann Dale', 'city':...",Sun Jun 11 09:38:07 UTC 1972,3.2
1,Lise Bartoletti,24,Jockey,34522,704,6767-5068-4665-5264-425,True,True,165.0,64.1,...,lise_bartoletti@hotmail.com,Female,False,B-,lise78,0.39,Christianity,"{'streetAddress': '80393 Spencer Rapids', 'cit...",Fri Sep 17 12:14:43 UTC 1999,2.1
2,Melodi Weber,44,Radiation Therapist,84363,554,3797-662581-41935,True,False,156.0,64.2,...,melodi@hotmail.com,Female,False,B-,melodi26,-0.03,Folk Religions,"{'streetAddress': '980 Cummings Lodge', 'city'...",Thu Oct 04 09:42:53 UTC 1979,3.3
3,Dalton Orn,21,Racehorse Trainer,54693,618,6771-8985-4318-4114,True,True,161.0,75.3,...,daltonorn@yahoo.com,Male,True,A-,dalton4,-0.52,Hinduism,"{'streetAddress': '159 Vickie Corners', 'city'...",Wed Nov 27 08:00:22 UTC 2002,1.6
4,Agnus Upton,34,Graphic Designer,79412,490,6533-6264-3114-7249-1499,False,False,155.0,55.3,...,agnus_upton@hotmail.com,Female,False,B+,agnus53,-0.65,Hinduism,"{'streetAddress': '0825 Noel Trail', 'city': '...",Fri Jan 26 00:20:56 UTC 1990,2.7


In [35]:
#getting a list of all the column names

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              10 non-null     object 
 1   age               10 non-null     int64  
 2   job               10 non-null     object 
 3   incomeUSD         10 non-null     int64  
 4   creditScore       10 non-null     int64  
 5   ccNumber          10 non-null     object 
 6   married           10 non-null     bool   
 7   hasChildren       10 non-null     bool   
 8   height            10 non-null     float64
 9   weight            10 non-null     float64
 10  eyeColor          10 non-null     object 
 11  email             10 non-null     object 
 12  gender            10 non-null     object 
 13  hasDegree         10 non-null     bool   
 14  bloodType         10 non-null     object 
 15  username          10 non-null     object 
 16  politicalLeaning  10 non-null     float64
 17  

Since we know we are only going to market this product to adults, we need to remove all children from the dataset 

In [36]:
#removing individuals under the age of 18 
df.drop(df[df['age'] < 18].index, inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              10 non-null     object 
 1   age               10 non-null     int64  
 2   job               10 non-null     object 
 3   incomeUSD         10 non-null     int64  
 4   creditScore       10 non-null     int64  
 5   ccNumber          10 non-null     object 
 6   married           10 non-null     bool   
 7   hasChildren       10 non-null     bool   
 8   height            10 non-null     float64
 9   weight            10 non-null     float64
 10  eyeColor          10 non-null     object 
 11  email             10 non-null     object 
 12  gender            10 non-null     object 
 13  hasDegree         10 non-null     bool   
 14  bloodType         10 non-null     object 
 15  username          10 non-null     object 
 16  politicalLeaning  10 non-null     float64
 17  

As we can see the file is now smaller and excludes anyone under the age of 18. 

Since the BMI API we are using requires height in inches and weight in pounds, we will need to update this in our dataframe as well. Currently, the Height is in CM and the Weight is in KG. 

Height: Convert from CM to In

Weight: Convert from KG to LB

To convert from CM to IN, I need to divide the Height column values by 2.54 in order to get the correct value in inches 
To convert from KG to LB, I need to multiply the Weight column values by 2.205 in order to get the correct value in pounds

In [37]:
#adding height in inches and weight in pounds
df['height_inches'] = df['height'] / 2.54

df['weight_pounds'] = df['weight'] * 2.205

In [38]:
#generating just the columns needed to get BMI
bmi_columns_needed = df[['age','height','weight','height_inches','weight_pounds']]
print(bmi_columns_needed)

   age  height  weight  height_inches  weight_pounds
0   52   161.0    54.8      63.385827       120.8340
1   24   165.0    64.1      64.960630       141.3405
2   44   156.0    64.2      61.417323       141.5610
3   21   161.0    75.3      63.385827       166.0365
4   34   155.0    55.3      61.023622       121.9365
5   31   175.0    60.2      68.897638       132.7410
6   28   166.0    59.4      65.354331       130.9770
7   26   166.0    65.3      65.354331       143.9865
8   34   161.0    74.3      63.385827       163.8315
9   36   176.0    74.6      69.291339       164.4930


### Appending in BMI Data

In [44]:
def calculate_bmi(row):
    url = "https://fitness-calculator.p.rapidapi.com/bmi"
    querystring = {
        "age": row['age'],
        "weight": row['weight'],
        "height": row['height']
    }
    headers = {
        "X-RapidAPI-Key": "4e03809b78msh7c9b407469fb512p1dd864jsnf85ade6acbd3",
        "X-RapidAPI-Host": "fitness-calculator.p.rapidapi.com"
    }
    response = requests.get(url, headers=headers, params=querystring)
    data = response.json()
    
    # Check if 'data' key is present in the API response
    if 'data' in data:
        bmi = data['data'].get('bmi', None)
        health = data['data'].get('health', None)
        healthy_bmi_range = data['data'].get('healthy_bmi_range', None)
        return bmi, health, healthy_bmi_range
    else:
        return None, None, None  # Return None in case of missing or unexpected data

# Add new columns 'bmi', 'health', and 'healthy_bmi_range' to the DataFrame
df['bmi'], df['health'], df['healthy_bmi_range'] = zip(*df.apply(calculate_bmi, axis=1))

# Display the updated DataFrame
print(df)

              name  age                  job  incomeUSD  creditScore  \
0   Rafael Kilback   52           Journalist      68750          475   
1  Lise Bartoletti   24               Jockey      34522          704   
2     Melodi Weber   44  Radiation Therapist      84363          554   
3       Dalton Orn   21    Racehorse Trainer      54693          618   
4      Agnus Upton   34     Graphic Designer      79412          490   
5     Arnold Doyle   31              Cashier      27380          586   
6   Octavio Walter   28    Best Boy Electric      58645          437   
7       Lela Sauer   26    Preschool Teacher      36627          610   
8     Joey Fritsch   34       Game Developer      93796          733   
9   Meghann Heller   36           Babysitter      20312          608   

                   ccNumber  married  hasChildren  height  weight  ...  \
0         3767-819973-23952     True         True   161.0    54.8  ...   
1   6767-5068-4665-5264-425     True         True   165.0  

Now that I have added in BMI, Health indicator, and the healthy BMI range, I would like to make my data set more robust to make insights easier. First, I will take the income column and add a new range that signifies lower, middle, or upper class. 

In [47]:
def categorize_income_level(incomeUSD):
    income = int(incomeUSD)  # Convert income to an integer
    
    if income < 52200:
        return 'Low income'
    elif 52200 <= income < 156600:
        return 'Middle income'
    else:
        return 'Upper income'

# Add a new column 'income level' to the DataFrame with the income level categories
df['income level'] = df['incomeUSD'].apply(categorize_income_level)

print(df)


              name  age                  job  incomeUSD  creditScore  \
0   Rafael Kilback   52           Journalist      68750          475   
1  Lise Bartoletti   24               Jockey      34522          704   
2     Melodi Weber   44  Radiation Therapist      84363          554   
3       Dalton Orn   21    Racehorse Trainer      54693          618   
4      Agnus Upton   34     Graphic Designer      79412          490   
5     Arnold Doyle   31              Cashier      27380          586   
6   Octavio Walter   28    Best Boy Electric      58645          437   
7       Lela Sauer   26    Preschool Teacher      36627          610   
8     Joey Fritsch   34       Game Developer      93796          733   
9   Meghann Heller   36           Babysitter      20312          608   

                   ccNumber  married  hasChildren  height  weight  ...  \
0         3767-819973-23952     True         True   161.0    54.8  ...   
1   6767-5068-4665-5264-425     True         True   165.0  